In [1]:
import dask.dataframe as dd
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
from dask.diagnostics import ProgressBar
from dask.multiprocessing import get
pbar=ProgressBar()
pbar.register()

In [3]:
%%time
import json


inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adresse_new_clean_reg': 'object',
    'Adress_new':'object',
    'INSEE': 'object',
    'Date_Début_Activité': 'object',
    'digit_inpi': 'object',
    'len_digit_address_inpi':'object',
    'Date_Greffe': 'object',
}


Wall time: 0 ns


In [4]:
inpi_col=['siren',
            'Code_Postal',
            'Code_Commune',
            'ncc',
            'Adress_new',
            'INSEE',
            'digit_inpi',
            'index',
            'Date_Greffe'
         ]
data_initial= dd.read_csv("temp/inpi_etb_stock_0.gz", 
                          usecols =inpi_col,
                          dtype=inpi_dtype,
                          compression='gzip',
                          blocksize=None,
                          low_memory=False,
                          infer_datetime_format=True
                      )

In [ ]:
data_initial.compute().shape

In [5]:
#contient 201705
columns_=['siren',
            'Code_Postal',
            'Code_Commune',
            'ncc',
            'Adress_new',
            'INSEE',
            'digit_inpi',
            'index',
            'Date_Greffe',
            'file_timestamp',
            'Libelle_Evt2'
         ]

data_evt_enriched= pd.read_csv("temp/201705_ETS_enriched_Normalized.gz", 
                      dtype=inpi_dtype,
                      compression='gzip',
                      low_memory=False,
                      #usecols=columns_
                      )
data_evt_enriched.shape

(24626, 44)

In [ ]:
#View columns to exclude from match
data_evt_enriched.isna().sum()

In [6]:
# Merge

match_var=['siren',
           'Code_Postal',
           'Code_Commune',
            'ncc',
            'Adress_new',
            'INSEE',
            'digit_inpi'
                     ]

temp=data_initial.merge(data_evt_enriched,
                   how='right',
                   indicator=True,
                    left_on=match_var,
                    right_on=match_var,
                    suffixes=['_initial', '_event']
                  ).compute()

[########################################] | 100% Completed |  1min 40.8s


In [7]:
# Merge stats
temp.groupby('_merge')['_merge'].count()

_merge
left_only         0
right_only     9185
both          18167
Name: _merge, dtype: int64

In [8]:
#right_only + both > len(data_evt_enriched)   => duplicates to remove
to_check=temp[temp['_merge'].isin(['both'])] #match, to keep, but there are duplicates
#no_match=temp[temp['_merge'].isin(['right_only'])] #no match at all, will be ignored

In [9]:
def split_duplication(df):
        """
        Split un dataframe si l'index (la variable, pas l'index) contient des
        doublons.
        L'idée est de distinguer les doublons resultants du merge avec l'INSEE
        Args:
        - df: Pandas dataframe contenant au moins une variable "index"
        Returns:
        - Un Dictionary avec:
            - not_duplication: Dataframe ne contenant pas les doublons
            - duplication: Dataframe contenant les doublons
            - report_dup: Une Serie avec le nombres de doublons
        """
        if 'count_duplicates_' in df.columns:
            df = df.drop(columns = 'count_duplicates_')

        df = df.merge(
            (df
                .groupby('index_event')['index_event']
                .count()
                .rename('count_duplicates_')
                .reset_index()
                )
                )
        try:
            df = df.compute()
        except:
            pass

        dic_ = {
            'not_duplication':df[df['count_duplicates_'].isin([1])],
            'duplication' : df[~df['count_duplicates_'].isin([1])],
            'report_dup':df[
            ~df['count_duplicates_'].isin([1])
            ]['count_duplicates_'].value_counts()
            }

        return dic_

In [10]:
#drop duplicates
test_dup=split_duplication(to_check)
#test_dup['not_duplication'].shape
#test_dup['not_duplication'].head()

In [17]:
test_dup['not_duplication'].head()

,Date_Greffe_initial,siren,ncc,Code_Postal,Code_Commune,Adress_new,INSEE,digit_inpi,index_initial,Code Greffe,...,len_digit_address_inpi,Siege_Domicile_Représentant,Activité_Ambulante,Activité_Saisonnière,Activité_Non_Sédentaire,file_timestamp,Libelle_Evt2,index_event,_merge,count_duplicates_
0,2017-05-03 00:00:00,829189455,MEXIMIEUX,01800,01244,RUE BOISSET BP 37,RUE,37,11.0,NaN,...,1,NaN,NaN,NaN,NaN,2017-05-17 07:36:28,nouveau_modifie,515,both,1
1,2017-05-02 00:00:00,500609011,SAINT MARTIN DU FRENE,01430,01373,1 IMPASSE DU TILLEUL,IMP,1,40.0,NaN,...,1,NaN,NaN,NaN,NaN,2017-05-12 21:47:11,nouveau_modifie,460,both,1
2,2017-05-02 00:00:00,500609011,SAINT MARTIN DU FRENE,01430,01373,1 IMPASSE DU TILLEUL,IMP,1,40.0,NaN,...,1,NaN,NaN,NaN,NaN,2017-05-12 21:47:11,nouveau_modifie,461,both,1
3,2015-08-10 00:00:00,800114803,PERONNAS,01960,01289,1 CHEMIN DE SAINT ROCH LE FLORENCE BATIMENT 1,CHE,1,61.0,NaN,...,2,NaN,NaN,NaN,NaN,2017-05-12 11:25:44,nouveau_modifie,349,both,1
4,2017-05-03 00:00:00,443620125,SAINT TRIVIER DE COURTES,01560,NaN,ROUTE DE PONT DE VAUX,RTE,NaN,70.0,NaN,...,0,NaN,NaN,NaN,NaN,2017-05-12 17:55:33,supprime,385,both,1


In [11]:
# List indexes of rows that are no duplicates
index_initial = test_dup['not_duplication']['index_initial']
index_event = test_dup['not_duplication']['index_event']

In [12]:
# Concat Initial and Evt
etape5= pd.concat([
    data_initial[data_initial['index'].isin(index_initial)].compute(),
    data_evt_enriched[data_evt_enriched['index'].isin(index_event)]
    ],
    axis=0,
    sort=True
    )

[########################################] | 100% Completed |  1min 30.0s


In [38]:
len(data_evt_enriched[data_evt_enriched['index'].isin(index_event)])

12724

In [40]:
data_evt_enriched[data_evt_enriched['index'].isin(index_event)].groupby('index')['index'].count().sort_values()

index
0        1
15625    1
15626    1
15627    1
15628    1
        ..
8479     1
8480     1
8481     1
8483     1
24608    1
Name: index, Length: 12724, dtype: int64

In [45]:
(data_evt_enriched[data_evt_enriched['index']
        .isin(index_event)]
        .groupby(['siren','file_timestamp','siren',
           'Code_Postal',
           'Code_Commune',
            'ncc',
            'Adress_new',
            'INSEE',
            'digit_inpi'])['index']
    .count()
    .sort_values()
    )

siren      file_timestamp       siren      Code_Postal  Code_Commune  ncc                 Adress_new                                    INSEE  digit_inpi
300773413  2017-05-31 07:41:36  300773413  92564        92063         RUEIL MALMAISON     22 RUE DES DEUX GARES                         RUE    22             1
798249751  2017-05-12 14:24:12  798249751  76200        76217         DIEPPE               105 RUE DE LA BARRE                          RUE    105            1
798237756  2017-05-12 17:58:51  798237756  44600        44184         SAINT NAZAIRE       7 RUE EMILE LITTREE                           RUE    7              1
798232526  2017-05-31 09:13:24  798232526  44850        44028         CELLIER             57 CHEMIN DU PERE FREIN                       CHE    57             1
798189676  2017-05-13 06:32:20  798189676  37100        37261         TOURS               86 RUE DE VILDE                               RUE    86             1
                                              

In [51]:
814844346  
data_evt_enriched[data_evt_enriched['siren'].isin(['827880683'])].to_excel('test.xlsx')

,Code Greffe,Nom_Greffe,Numero_Gestion,RCS_Registre,Date_Greffe,Libelle_Evt,ID_Etablissement,siren,Nom_Commercial,Enseigne,...,digit_inpi,list_digit_inpi,len_digit_address_inpi,Siege_Domicile_Représentant,Activité_Ambulante,Activité_Saisonnière,Activité_Non_Sédentaire,file_timestamp,Libelle_Evt2,index
8264,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8264
8265,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8265
8266,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8266
8267,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8267
8268,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8268
8269,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8269
8270,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8270
8271,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8271
8272,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8272
8273,NaN,NaN,NaN,NaN,2017-05-29,Modifications relatives à un établissement,2,827880683,NaN,NaN,...,1,NaN,1,NaN,NaN,NaN,NaN,2017-05-30 08:20:23,nouveau_modifie,8273


In [50]:
dtype_2={'Siren': 'object',
        'Type': 'object',
        'Adresse_Ligne1': 'object',
        'Adresse_Ligne2': 'object',
        'Adresse_Ligne3': 'object',
        'Code_Postal': 'object',
        'Ville': 'object',
        'Code_Commune': 'object',
        'ID_Etablissement': 'object',
        'Date_Greffe': 'object',
        'Libelle_Evt': 'object',
        'Code Greffe': 'object',
        'Domiciliataire_Siren': 'object',
       }

data_evt= pd.read_csv("temp/2017_EVT_ETS.gz", 
                      dtype=dtype_2,
                      compression='gzip',
                      low_memory=False
                      )
data_evt[data_evt['Siren'].isin(['827880683'])]

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type,Siège_PM,RCS_Registre,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,...,Activité_Saisonnière,Activité_Non_Sédentaire,Date_Début_Activité,Activité,Origine_Fonds,Origine_Fonds_Info,Type_Exploitation,ID_Etablissement,Date_Greffe,Libelle_Evt


In [13]:
etape5.head()

,Activité,Activité_Ambulante,Activité_Non_Sédentaire,Activité_Saisonnière,Adress_new,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,Adresse_new_clean_reg,Code Greffe,...,Ville,count_initial_inpi,digit_inpi,file_timestamp,index,len_digit_address_inpi,list_digit_inpi,ncc,possibilite,siren
11,NaN,NaN,NaN,NaN,RUE BOISSET BP 37,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37,NaN,11,NaN,NaN,MEXIMIEUX,NaN,829189455
40,NaN,NaN,NaN,NaN,1 IMPASSE DU TILLEUL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,40,NaN,NaN,SAINT MARTIN DU FRENE,NaN,500609011
61,NaN,NaN,NaN,NaN,1 CHEMIN DE SAINT ROCH LE FLORENCE BATIMENT 1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,61,NaN,NaN,PERONNAS,NaN,800114803
70,NaN,NaN,NaN,NaN,ROUTE DE PONT DE VAUX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,70,NaN,NaN,SAINT TRIVIER DE COURTES,NaN,443620125
71,NaN,NaN,NaN,NaN,1476 ROUTE DE LYON,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1476,NaN,71,NaN,NaN,BALAN,NaN,510957269


In [20]:
(etape5.sort_values(by=['siren',
                       'Date_Greffe',
                       'file_timestamp'])
        .to_excel('etape5.xlsx')
)

In [22]:
match_var

['siren',
 'Code_Postal',
 'Code_Commune',
 'ncc',
 'Adress_new',
 'INSEE',
 'digit_inpi']

In [21]:
# Vérif qu'on a bien à chaque fois seulement 1 initial et 1 evt (count=2)

(etape5
 .groupby(match_var)['siren']
 .count()
 .rename('nb_rows')
 .reset_index()
 .groupby('nb_rows')['siren']
 .count()
)

nb_rows
2     4006
3     1096
4      324
5      137
6       74
7       17
8        5
9        2
10       1
11       3
Name: siren, dtype: int64

In [31]:
# Un ex qui a count>2 : '301853172'
etape5.groupby(match_var)[['siren']].count()
# Il y a bien 2 evt. Toutes les colonnes de match identiques, index evt different.

siren
siren     Code_Postal Code_Commune ncc                    Adress_new                                       INSEE digit_inpi       
300773413 92564       92063        RUEIL MALMAISON        22 RUE DES DEUX GARES                            RUE   22              2
301853172 35400       35288        SAINT MALO              53 RUE DU GRAND JARDIN  ZAC DE LA MOINERIE      RUE   53              3
302043856 94400       94081        VITRY SUR SEINE        7 RUE DE L ABBE ROGER DERR                       RUE   7               2
302475041 06200       06088        NICE                   63 73 ROUTE DE GRENOBLE                          RTE   63              2
          06250       06085        MOUGINS                235 ROUTE DU CANNET  QUARTIER DE LA NARTASSIERE  RTE   235             2
...                                                                                                                            ...
967504945 94430       94019        CHENNEVIERES SUR MARNE 14 RUE GAY LUSSAC                                RUE   14              2
969502004 26400       26108        CREST                   19 AVENUE ADRIEN FAYOLLE                        AV    19              2
          38100       38185        GRENOBLE                32 AVENUE RHIN ET DANUBE                        AV    32              2
977150242 11100       11262        NARBONNE                4 BOULEVARD DOCTEUR FERROUL                     BD    4               4
998823504 92340       92014        BOURG LA REINE         81 BOULEVARD DU MARECHAL JOFFRE                  BD    81              2

[5665 rows x 1 columns]

In [ ]:
# STEP 2 of algorithm
# Matching with less columns

In [36]:
data_initial_step2 = data_initial[~data_initial['index'].isin(index_initial)].compute()

[########################################] | 100% Completed |  1min 53.3s


In [37]:
# Merge

match_var=['siren',
           'Code_Postal',
           'Code_Commune',
            'ncc',
            'Adress_new',
            'INSEE'
          ]

temp=data_initial_step2.merge(data_evt_enriched,
                   how='right',
                   indicator=True,
                    left_on=match_var,
                    right_on=match_var,
                    suffixes=['_initial', '_event']
                  )

AttributeError: 'DataFrame' object has no attribute 'compute'